In [72]:
%matplotlib inline

import os
import sys
import random
import imp


import numpy as np
import pandas as pd
import xarray as xr


import climatools.aerosol.aerosol as climaaerosol
import climatools.io.cesm as climaio
import climatools.aerosol.aerosol_constants as aeroconst
import climatools.rrtmg.column_model as rrtmg_column
import climatools.units as climaunits


imp.reload(climaaerosol)
imp.reload(climaio)
imp.reload(aeroconst)
imp.reload(rrtmg_column)
imp.reload(climaunits)

<module 'climatools.units' from '/nuwa_cluster/home/jackyu/climatools/climatools/units.py'>

### General workflow

In [48]:
## where are the CAM history files
dir_camhist = '/scratch/lupin/cesm1/archive/f19_g16.B1850.std/atm/hist'

In [49]:
## Prep o3 concentration data

ds_o3 = climaaerosol.OZONE_DATASET
da_o3 = ds_o3['O3']

# interpolate o3 concentration along lev
levinterpfunc_da_o3 = climaaerosol.get_interpfunc(da_o3, dim='lev')

# evaluate o3 concentration at the 30 levs of CAM history
filepath_camhist = os.path.join(dir_camhist,
                                random.choice(os.listdir(dir_camhist)))
ds_camhist = xr.open_dataset(filepath_camhist, decode_cf=False)

da_o3_levinterp = levinterpfunc_da_o3(coords=ds_camhist.coords['lev'])


# interpolate o3 concentration along time
timeinterpfunc_o3 = climaaerosol.get_interpfunc(da_o3_levinterp,
                                                dim='time')

In [73]:
aerosol = True

print("load a CAM history file from some previous global run")
ds = climaio.load_camhistory(readfrom='f19_g16.B1850.std.cam2.h0.0001-06.nc')

print("convert hybrid levels/layers to pressure levels/layers")
ds = climaunits.hybrid2mbar(ds)

print("get ozone concentration for the time in this CAM history file")
ds = climaaerosol.get_o3_concentration(ds, interpfunc=timeinterpfunc_o3)

print("get o2 concentration")
ds = climaaerosol.get_o2_concentration(ds)

print("interpolate layer values to levels for: T, Q and O3")
ds = climaaerosol.interp_layers2levels(ds, vars=['T', 'Q', 'O3'])

if aerosol:
    print('You asked for aerosol. You got it.')
    # gather aerosol species mmr into a new variable in the dataset
    ds = climaaerosol.aerosol_species_mmr(ds)

    # aerosol water uptake. computes qaerwat, dgncur_awet, wetdens. add to ds
    ds = climaaerosol.wateruptake(ds)
    
    # get pressure difference for layer mass
    ds = climaaerosol.get_pressure_difference(ds)

    # compute and add to ds modal aerosol optical properties 
    ds = climaaerosol.modal_aero_sw(ds)

stackdims = ('time', 'lat', 'lon')

cols = ds['RELHUM'].stack(cols=stackdims).coords['cols']

print()

for time, lat, lon in cols.values[:2]:
    print("Working on column at time {}, lat {}, lon {}".format(time, lat, lon))
    print("write input tapes for RRTMG column model")
    rrtmg_column.write_sw_inputfiles(ds,
                                     time=time, lat=lat, lon=lon,
                                     aerosol=aerosol)

    print("execute RRTMG")
#    assert os.system('./rrtmg_lksdokw') == 0

    print("this dataset contains things like flux down, up and heating rates")
#    ds_rrtmg_col = climarrtmg.load_OUTPUT_RRTM(readfrom='OUTPUT_RRTM', 
#                                               signed_fluxes=True)
#    ds_rrtmg.loc[dict(time=time, lat=lat, lon=lon)] = ds_rrtmg_col

    print("write RRTMG output to netCDF file")
#    ds_rrtmg.to_netcdf('RRTMG_f19_g16.B1850.std.cam2.h0.0001-06.nc')
    
    print()




load a CAM history file from some previous global run
convert hybrid levels/layers to pressure levels/layers
get ozone concentration for the time in this CAM history file
get o2 concentration
interpolate layer values to levels for: T, Q and O3
You asked for aerosol. You got it.

Working on column at time 181.0, lat -90.0, lon 0.0
write input tapes for RRTMG column model
execute RRTMG
this dataset contains things like flux down, up and heating rates
write RRTMG output to netCDF file

Working on column at time 181.0, lat -90.0, lon 2.5
write input tapes for RRTMG column model
execute RRTMG
this dataset contains things like flux down, up and heating rates
write RRTMG output to netCDF file



In [45]:
!cat INPUT_RRTM

$ CXID                                                                          
                  10                             1                                1     98   00   00
            181   60.0000                                                                           
           0  0                                                                      
    0       -31         0    7    0                                             
   672.642     2.255
   672.642   662.628   652.110   640.076   626.628   611.881   595.955   578.979
   549.657   510.204   463.451   412.512   361.592   318.310   281.520   250.247
   223.665   201.070   181.863   154.587   131.401   111.693    94.941    80.701
    63.323    45.867    30.669    18.555    10.158     5.032     2.255        
  -31(lat,lon) = (-90.0, 2.5)
 0.000e+00 6.726e+02 2.117e+02     AA   CAAAAAC
-1.410e-05 2.847e-04 2.320e-08 2.757e-07 0.000e+00 7.916e-07 2.314e-01
 0.000e+00 6.626e+02 2.254e+02     AA   CAAAAAC
 8.418e-05

In [18]:
def write_sw_inputfiles(ds, 
                        time=181, lat=-90, lon=0, 
                        aerosol=False):
    
    write_input_rrtm(ds, 
                     time=time, lat=lat, lon=lon, 
                     aerosol=aerosol)
    if aerosol:
        write_in_aer_rrtm(ds, time=time, lat=lat, lon=lon)

        
def write_input_rrtm(ds, 
                     time=181, lat=-90, lon=0,
                     aerosol=False):
    if aerosol:
        iaer = 10
    else:
        iaer = 0
    
    content = collections.deque([])
    
    content.append(swin.record_1_1(cxid))
    content.append(swin.record_1_2(iaer=iaer))
    ...
    with open('INPUT_RRTM', mode='w', encoding='utf-8') as file:
        file.write('\n'.join(content))
        

def write_in_aer_rrtm(ds, time=181, lat=-90, lon=0):
    
    content = collections.deque([])
    
    # record A1.1
    content.append(swin.record_a1_1(naer=1))
    
    # record A2.1
    content.append(swin.record_a2_1(nlay=,
                                    iaod=1,
                                    issa=1,
                                    ipha=1))
    
    # record A2.1.1 for all layers
    for lev in ds.coords['lev']:
        lay = lev2lay(lev)
        aod = ds['tauxar'].sel(time=time, lat=lat, lon=lon, lev=lev)
        content.append(swin.record_a2_1_1(lay=lay, aod=aod))
    
    # record A2.2
    ssa = ds['wa'].sel(time=time, lat=lat, lon=lon).isel(lev=0)
    content.append(swin.record_a2_2(ssa=ssa))
    
    # record A2.3
    phase = ds['ga'].sel(time=time, lat=lat, lon=lon).isel(lev=0)
    content.append(swin.record_a2_3(phase=phase))
    
    with open('IN_AER_RRTM', mode='w', encoding='utf-8') as file:
        file.write('\n'.join(content))
    
    

SyntaxError: invalid syntax (<ipython-input-18-3b280e15781a>, line 37)

In [200]:
def func1(a, b):
    return a + b


def method1(ds):
    ds = ds.copy(deep=True)
    ds['C'] = func1(ds['A'], ds['B'])
    return ds
    

def method2(ds):
    ds = ds.copy(deep=True)
    ds['C'] = xray.DataArray.from_series(
        ds.to_dataframe().apply(lambda x: func1(x['A'], x['B']), axis=1))
    return ds


In [291]:
nx1, nx2, nx3 = 5, 3, 6

ds = xray.Dataset()
ds.coords.update({'x1': ('x1', range(nx1)), 
                  'x2': ('x2', range(nx2)), 
                  'x3': ('x3', range(nx3))})

ds['A'] = (['x1', 'x2', 'x3'], np.arange(nx1*nx2*nx3).reshape((nx1, nx2, nx3)))
ds['B'] = (['x1', 'x2', 'x3'], np.arange(nx1*nx2*nx3).reshape((nx1, nx2, nx3)) + 100) 



In [290]:

def method1(ds):
    
    def func(a, b):
        a = np.roll(a, 1, axis=2)
        b = np.fft.fft(b, axis=2)
        return a + b
    
    ds = ds.copy(deep=True)
    ds.update({'C': (['x1', 'x2', 'x3'], func(ds['A'], ds['B']))})
    return ds


def method2(ds):
    ds = ds.copy(deep=True)
    df = ds.to_dataframe()
    
    srsAA = srsA.unstack(2)\
    .apply(lambda x: np.roll(x, 1), axis=1).stack()

    srsBB = srsB.unstack(2)\
    .apply(lambda x: np.fft.fft(x), axis=1).stack()

    srsC = srsAA + srsBB

    df = pd.concat([df['A'], df['B'], srsC], 
                   axis=1, keys=['A', 'B', 'C'])

    ds = xray.Dataset.from_dataframe(df)
    return ds
  

In [292]:
ds

<xray.Dataset>
Dimensions:  (x1: 5, x2: 3, x3: 6)
Coordinates:
  * x2       (x2) int64 0 1 2
  * x1       (x1) int64 0 1 2 3 4
  * x3       (x3) int64 0 1 2 3 4 5
Data variables:
    A        (x1, x2, x3) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
    B        (x1, x2, x3) int64 100 101 102 103 104 105 106 107 108 109 110 ...

In [293]:
ds1 = method1(ds)
ds2 = method2(ds)

In [294]:
ds1

<xray.Dataset>
Dimensions:  (x1: 5, x2: 3, x3: 6)
Coordinates:
  * x2       (x2) int64 0 1 2
  * x1       (x1) int64 0 1 2 3 4
  * x3       (x3) int64 0 1 2 3 4 5
Data variables:
    A        (x1, x2, x3) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
    B        (x1, x2, x3) int64 100 101 102 103 104 105 106 107 108 109 110 ...
    C        (x1, x2, x3) complex128 (620+0j) (-3+5.19615242271j) ...

In [295]:
ds2

<xray.Dataset>
Dimensions:  (x1: 5, x2: 3, x3: 6)
Coordinates:
  * x1       (x1) int64 0 1 2 3 4
  * x2       (x2) int64 0 1 2
  * x3       (x3) int64 0 1 2 3 4 5
Data variables:
    A        (x1, x2, x3) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
    B        (x1, x2, x3) int64 100 101 102 103 104 105 106 107 108 109 110 ...
    C        (x1, x2, x3) complex128 (620+0j) (-3+5.19615242271j) ...

In [296]:
all(ds1 == ds2)

True

In [300]:
%timeit method1(ds)

1000 loops, best of 3: 1.4 ms per loop


In [299]:
%timeit method2(ds)

10 loops, best of 3: 20.1 ms per loop


### Modal_aero_wateruptake()